In [2]:
import datetime
import openpyxl
import glob
import os
import re
import pprint
import math
import numpy as np

In [14]:
dialog_list = []
now = datetime.datetime.now()

for xlsx_file in xlsx_files:
    # エクセルブック
    wb = openpyxl.load_workbook(xlsx_file, data_only=True)

    # エクセルシート
    ws = wb["Transcription"]

    # ヘッダーを除いて順番に読み込む
    for row in ws["H4:H1000"]: 
        if row[0].value is None:
            # print()
            break
        dialog_list.append(row[0].value)
        dialog = '\n'.join(dialog_list)

# 余計な文字をカット
# =:削除, {#}:中の文字ごと削除, <#>:中の文字ごと削除, (#):中の文字ごと削除, [#]:中の文字ごと削除, 【【:削除, 】】:削除
# JM or JF + 数字3桁: 削除
dialog = re.sub(r'=','',dialog)
dialog = re.sub(r'{.+?}','',dialog)
dialog = re.sub(r'\[.+?\]','',dialog)
dialog = re.sub(r'《.+?》','',dialog)
dialog = re.sub(r'\(.+?\)','',dialog)
dialog = re.sub(r'<.+?>','',dialog)
dialog = re.sub(r'[.+?]','',dialog)
dialog = re.sub(r'【【','',dialog)
dialog = re.sub(r'】】','',dialog)
dialog = re.sub(r'\\u3000','',dialog) 
dialog = re.sub(r'JM[0-9]{3}','',dialog)
dialog = re.sub(r'JF[0-9]{3}','',dialog)
dialog = re.sub(r'。。','。',dialog)
dialog = re.sub(r'。、','',dialog)

# 分割個数分のdialog.txtを作成
f = open('dialog(full).txt', 'w', encoding="utf-8")
f.write(dialog)

2174153

In [170]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
import MeCab
import random
import sys
import io
%matplotlib inline

# path = './dialog(full)_mini.txt'
path = './dialog(full).txt'

with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
# print('corpus length:', len(text))
text = re.sub(r'「|」|『|』','',text)
text = re.sub(r'人名|施設|地名|場所|','',text)
text = re.sub(r'。|,|、|#|ー','',text)
print(text)

# 形態素で単語単位に分割
tagger = MeCab.Tagger('-Owakati')

# 単語をインデックスに変換
text = tagger.parse(text).split(' ')

chars = text
count = 0
char_indices = {}  # 辞書初期化
indices_char = {}  # 逆引き辞書初期化
 
for word in chars:
    if not word in char_indices:  # 未登録なら
       char_indices[word] = count  # 登録する      
       count +=1
       print(count,word)  # 登録した単語を表示

# 逆引き辞書を辞書から作成する
indices_char = dict([(value, key) for (key, value) in char_indices.items()])

maxlen = 5
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
# print(sequences)

608 賢一
19609 釈由美子
19610 せんな
19611 俳優座
19612 別役
19613 みのり
19614 存じ
19615 不条理
19616 レベ
19617 邦画
19618 洋画
19619 ロドショ
19620 netflix
19621 amazon
19622 プライム
19623 ワンパンマン
19624 サイタマ
19625 パンチ
19626 ゃいけないんでしたっけ
19627 お話し
19628 埼
19629 らっしゃら
19630 今中
19631 応球
19632 見守っ
19633 べつ
19634 セリフ
19635 再試
19636 ロドオブザリング
19637 温かい
19638 ボボヘミアンラプソディ
19639 最多
19640 響か
19641 レディガガ
19642 賛否
19643 百田
19644 尚樹
19645 フォルトゥナ
19646 モンスタ
19647 幅広
19648 pop
19649 ボカロ
19650 なにやら
19651 アティ
19652 king
19653 prince
19654 シックス
19655 キッズ
19656 キンキ
19657 昔ながら
19658 邦楽
19659 福山
19660 雅治
19661 カウントダウン
19662 シラン
19663 ペラッペラ
19664 ノミ
19665 症候群
19666 ゴホンゴホンゲホンゲホン
19667 薬効
19668 鍼
19669 ステロイド
19670 ピタッ
19671 漢方
19672 配合
19673 調合
19674 カプセル
19675 錠剤
19676 軟膏
19677 塗り薬
19678 ょっとじゅなんだろう
19679 道行く
19680 高み
19681 マナ
19682 高望み
19683 ゃなければぜんぜん
19684 sta
19685 ゃなでもめんどくさいや
19686 あれよあれよ
19687 小売
19688 不動産
19689 システムエンジニア
19690 基底
19691 染みれ
19692 にぶん
19693 文理
19694 アレルギ
19695 恐怖
19696 くせ者
19697 p
19698 忘れ物
19699 択
19700 逆算
19701 乗り切っ
197

In [175]:
import numpy as np
import gensim
import string

print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)

# 埋め込みモデルの保存
word_model.wv.save_word2vec_format("./corpus.vec.pt", binary=True)

# 全単語のベクトルを代入
pretrained_weights = word_model.wv.syn0

vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)


Training word2vec...
Result embedding shape: (21405, 100)
ipykernel_launcher:12: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


In [141]:
# 辞書の作成
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

In [178]:
print('\nPreparing the data for LSTM...')
# train_x = np.zeros([len(sentences), maxlen], dtype=np.int8)
# train_y = np.zeros([len(sentences)], dtype=np.int8)
train_x = np.abs(np.zeros([len(sentences), maxlen], dtype=np.uint8))
train_y = np.abs(np.zeros([len(sentences)], dtype=np.uint8))
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (941703, 5)
train_y shape: (941703,)


In [157]:
train_x

array([[ 66,  15,  21, 340,   0],
       [ 15,  21, 340, 115,   0],
       [ 21, 340, 115, 164,   0],
       ...,
       [222,   7,  41,  95,   0],
       [  7,  41,  95,  17,   0],
       [ 41,  95,  17,  50,   0]])

In [144]:
batch_size = 32

# コーパスが巨大でありMemmoryErrorが発生するため、ジェネレータでミニバッチにわけて訓練する
def batch_iter(data, labels, batch_size, shuffle=True):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1

    def data_generator():
        data_size = len(data)
        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                X = shuffled_data[start_index: end_index]
                y = shuffled_labels[start_index: end_index]
                yield X, y

    return num_batches_per_epoch, data_generator()

train_steps, train_batches = batch_iter(train_x, train_y, batch_size)

In [145]:
# Embedding層を追加
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [168]:
# 確率的に次の単語を選ぶ関数
def sample(preds, temperature=1.0):
    preds = np.log(preds) / temperature
    dist = np.exp(preds)/np.sum(np.exp(preds))
    choices = range(len(preds))
    return np.random.choice(choices, p=dist)

def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [147]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
 
    # start_index = random.randint(0, len(text) - maxlen - 1)
    start_index = 0  # テキストの最初からスタート
    for diversity in [0.2]:  # diversity は 0.2のみ使用 
        print('----- diversity:', diversity)
 
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        # sentence はリストなので文字列へ変換して使用
        generated += "".join(sentence)
        print(sentence)
        
        # sentence はリストなので文字列へ変換して使用
        print('----- Generating with seed: "' + "".join(sentence)+ '"')
        sys.stdout.write(generated)
 
        for i in range(400):
 
            preds = model.predict(train_x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
 
            generated += next_char
            sentence = sentence[1:]
            # sentence はリストなので append で結合する
            sentence.append(next_char)  
 
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
 
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [179]:
from keras.callbacks import EarlyStopping

# EarlyStopping
es_cb = EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

# コーパスをFullの半分以下程度にして訓練するよう修正
history = model.fit(train_x, train_y,
                    batch_size=32,
                    epochs=1,
                    callbacks=[print_callback, es_cb])

Epoch 1/1
941703/941703 [==============================] - 6069s 6ms/step - loss: 4.3431

----- Generating text after Epoch: 0
----- diversity: 0.2
['する', 'うん', 'あ', '昨日', 'なか']
----- Generating with seed: "するうんあ昨日なか"
するうんあ昨日なかうん昨日昨日うん昨日昨日昨日昨日うんうん3うん昨日うん昨日昨日うんなか昨日昨日昨日うん昨日来昨日なかなか昨日うん昨日昨日昨日昨日昨日昨日昨日昨日昨日昨日昨日うん昨日うん昨日

KeyboardInterrupt: 

In [ ]:
# 損失の推移
loss = history.history["loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "bo", label = "Training loss" )
plt.title("Training loss")
plt.legend()
plt.show()
plt.savefig("loss_by_word2vec.png")
plt.close()

In [19]:
# モデルと学習結果（パラメータ）の保存
json_string = model.to_json()
open('dialog_model_by_word2vec.json', 'w').write(json_string)
model.save('dialog_model_by_word2vec.h5')